In [76]:
import numpy as np
import pandas as pd

In [77]:
import os
import sys

In [78]:
import datetime
from datetime import timezone

In [79]:
import wave

In [84]:
import tensorflow as tf

In [80]:
path = "./rawdata/samples.csv"
df = pd.read_csv(path, usecols=[0, 1, 2, 3, 4], names=['date', 'time', 'temp', 'hum', 'audio'])
df.head()

,date,time,temp,hum,audio
0,18/10/2020,9:40:10,21,65,audio2.wav
1,18/10/2020,9:41:12,21,65,audio3.wav
2,18/10/2020,9:42:14,21,65,audio4.wav
3,18/10/2020,9:43:16,21,65,audio1.wav
4,18/10/2020,9:44:18,21,65,audio2.wav


In [99]:
#create the tfrecord dataset and fill it in with the data
out_path = './samples.tfrecord'

def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

with tf.io.TFRecordWriter(out_path) as writer:
            for date, time, filename, temp, hum in zip(df.date, df.time, df.audio, df.temp, df.hum):
                year = int(date.split('/')[2])
                month = int(date.split('/')[1])
                day = int(date.split('/')[0])
                second = int(time.split(':')[2])
                minute = int(time.split(':')[1])
                hour = int(time.split(':')[0])

                dt = datetime.datetime(year, month, day, hour, minute, second, 0, tzinfo=timezone.utc)
                timestamp = int(dt.timestamp())
                
                file_path = './rawdata/' + str(filename)
                raw_audio = tf.io.read_file(file_path)
                audio, sample_rate = tf.audio.decode_wav(
                    raw_audio,
                    desired_channels=1)  # mono
                
                fp = wave.open(file_path)
                nchan = fp.getnchannels()
                N = fp.getnframes()
                data = fp.readframes(N * nchan)
                

                datetime_feature = tf.train.Feature(int64_list = tf.train.Int64List(value=[timestamp]))
                temp_feature = tf.train.Feature(int64_list = tf.train.Int64List(value=[temp]))
                hum_feature = tf.train.Feature(int64_list = tf.train.Int64List(value=[hum]))
                audio_feature = tf.train.Feature(float_list = tf.train.FloatList(value=audio.numpy().flatten().tolist()]))
                
                mapping = {
                    'datetime' : datetime_feature,
                    'temperature' : temp_feature,
                    'humidity' : hum_feature,
                    'audio' : audio_feature
                }
                
                feature_map = tf.train.Features(feature=mapping)
                
                example = tf.train.Example(features=feature_map)

                writer.write(example.SerializeToString())

TypeError: [9.1552734375e-05, -0.001220703125, -0.002593994140625, -0.00494384765625, -0.0064697265625, -0.0062 has type list, but expected one of: int, long, float

In [75]:
#convert audio into raw data
#alternative to tf.audio

samples = []

for filename in df.audio:
    path = './rawdata/' + str(filename)
    fp = wave.open(path)
    nchan = fp.getnchannels()
    N = fp.getnframes()
    data = fp.readframes(N * nchan)
    samples.append(data)


